In [2]:
!pip install seaborn
#import all libraries and dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

     |████████████████████████████████| 296kB 23.0MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.

In [7]:
world_cup = pd.read_csv('World Cup 2019 Dataset.csv')
results = pd.read_csv('results.csv')

In [8]:
#narrowing to team patcipating in the world cup
worldcup_teams = ['England', ' South Africa', '', 'West Indies', 
            'Pakistan', 'New Zealand', 'Sri Lanka', 'Afghanistan', 
            'Australia', 'Bangladesh', 'India']

In [9]:
#cleaning date column for exploration
results.loc[results.date == 'Jul 3-4, 2010', 'date'] = '03-07-2010'
results.loc[results.date == 'Aug 19-20, 2010', 'date'] = '19-08-2010'
results.loc[results.date == 'Jul 11-12, 2012', 'date'] = '11-07-2013'
results.loc[results.date == 'Jul 2-3, 2013', 'date'] = '02-07-2013'
results.loc[results.date == 'Jul 7-8, 2013', 'date'] = '03-07-2013'
results.loc[results.date == 'Aug 27-28, 2013', 'date'] = '27-08-2013'
results.loc[results.date == 'Dec 10-11, 2014', 'date'] = '10-12-2014'

In [23]:
df_teams_final  = results[(results['Team_1'].isin(worldcup_teams)) & (results['Team_2'].isin(worldcup_teams))]

In [10]:
#df_teams_1 = results[results['Team_1'].isin(worldcup_teams)]
#df_teams_2 = results[results['Team_2'].isin(worldcup_teams)]
#df_teams = pd.concat((df_teams_1, df_teams_2))
#df_teams.drop_duplicates()

,date,Team_1,Team_2,Winner,Margin,Ground
0,4-Jan-10,Bangladesh,Sri Lanka,Sri Lanka,7 wickets,Dhaka
1,5-Jan-10,India,Sri Lanka,Sri Lanka,5 wickets,Dhaka
2,7-Jan-10,Bangladesh,India,India,6 wickets,Dhaka
3,8-Jan-10,Bangladesh,Sri Lanka,Sri Lanka,9 wickets,Dhaka
4,10-Jan-10,India,Sri Lanka,India,8 wickets,Dhaka
...,...,...,...,...,...,...
933,21-May-17,Ireland,New Zealand,New Zealand,190 runs,Dublin (Malahide)
940,3-Jun-17,South Africa,Sri Lanka,South Africa,96 runs,The Oval
986,15-Oct-17,South Africa,Bangladesh,South Africa,10 wickets,Kimberley
988,18-Oct-17,South Africa,Bangladesh,South Africa,104 runs,Paarl


In [25]:
year = []
for row in df_teams_final['date']:
    year.append(int(row[7:]))
df_teams_final['match_year']= year
df_teams_2011 = df_teams_final[df_teams_final.match_year >= 11]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [26]:
#dropping columns that wll not affect match outcomes
df_teams_2011 = df_teams_2011.drop(['date','Margin', 'Ground','match_year'], axis=1)
df_teams_2011.head()

,Team_1,Team_2,Winner
144,Australia,England,Australia
146,Australia,England,Australia
148,New Zealand,Pakistan,New Zealand
149,Australia,England,Australia
151,New Zealand,Pakistan,no result


In [27]:
#Building the model
#the prediction label: The winning_team column will show "1" Team 1 has won and "2" if the away team has won.

df_teams_2011 = df_teams_2011.reset_index(drop=True)
df_teams_2011.loc[df_teams_2011.Winner == df_teams_2011.Team_1,'winning_team']=1
df_teams_2011.loc[df_teams_2011.Winner == df_teams_2011.Team_2, 'winning_team']=2
df_teams_2011 = df_teams_2011.drop(['winning_team'], axis=1)

df_teams_2011.head()

,Team_1,Team_2,Winner
0,Australia,England,Australia
1,Australia,England,Australia
2,New Zealand,Pakistan,New Zealand
3,Australia,England,Australia
4,New Zealand,Pakistan,no result


In [28]:
#convert team-1 and team-2 from categorical variables to continous inputs 
# Get dummy variables
final = pd.get_dummies(df_teams_2011, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

# Separate X and y sets
X = final.drop(['Winner'], axis=1)
y = final["Winner"]


# Separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)

In [29]:
final.shape

(340, 19)